##### imports

In [859]:
from numpy import exp
from numpy import sqrt
from numpy import cos
from numpy import e
from numpy import pi
from typing import List, Callable, Tuple
import random

#### Definindo váriaveis e função de avaliação

In [860]:
A = 20
B = 0.2
C = 2 * pi
D = 30

In [861]:
TAM_DA_POPULACAO = 10
POPULACAO = []
QTD_DE_FILHOS = 7 * TAM_DA_POPULACAO
TAXA_APRENDIZAGEM = 1 / sqrt(D)
EPSILON_0 = 0.001
EPSILON_1 = 10
MAX_ITERS = 10_000
CURR_SUCCESS = 0

In [862]:
def ackley_funct(x: List[float]):
  return - A * exp(-B * sqrt(sum(map(lambda n: n ** 2, x)) / D)) \
          - exp(sum(map(lambda n: cos(C * n), x)) / D) \
          + A + exp(1)

In [863]:
def sum_squares_function(x: List[float]):
    soma = 0

    for i in range(D):
        soma += (i+1) * x[i] ** 2

    return soma

In [864]:
FITNESS_FUNC = sum_squares_function

In [865]:
def fitness(fitness_func: Callable, cromossome: List[float]):
    return fitness_func(cromossome)

In [866]:
print(f'{ackley_funct([0 for _ in range(30)]) : .15f}')

 0.000000000000000


# Estratégia Evolutiva

## Versão 1

### Inicialização da população

In [867]:
def generate_individual():
  x = []
  for i in range(D):
    x.append(random.uniform(-15, 15))
    
  return x

In [868]:
def generate_population():
  global POPULACAO, TAM_DA_POPULACAO
  
  for _ in range(TAM_DA_POPULACAO):
    POPULACAO.append((generate_individual(), random.normalvariate(0, 1)))

In [869]:
def reset_population() -> None:
    global POPULACAO

    POPULACAO = []

### Seleção de pais

In [870]:
def select_parents():
    global POPULACAO

    pai1, pai2 = random.sample(POPULACAO, 2)

    return pai1, pai2

### Recombinação

In [871]:
# Local discreto
def evolutive_crossover(pai1: Tuple[List[float], float], pai2: Tuple[List[float], float]):
    filho = []
    for i in range(D):
        #filho += random.sample([pai1[0][i], pai2[0][i]], 1)
        filho += [(pai1[0][i] + pai2[0][i])/2]
        
    #sigma = random.sample([pai1[1], pai2[1]], 1)[0]
    sigma = (pai1[1] + pai2[1])/2

    return filho, sigma

In [872]:
def crossover():
    global QTD_DE_FILHOS

    filhos = []
    for _ in range(QTD_DE_FILHOS):
        pai1, pai2 = select_parents()
        filhos.append(evolutive_crossover(pai1, pai2))

    return filhos

### Mutação

In [873]:
def regra_1_quinto(sigma: float, sucesso: float) -> float:
    c = 0.8
    
    if sucesso < 1 / 5:
        sigma = sigma / c
    elif sucesso > 1 / 5:
        sigma = sigma * c
    
    return sigma

In [874]:
def calc_sigma(sigma: float) -> float:
    global TAXA_APRENDIZAGEM, EPSILON_0

    new_sigma = sigma * exp(TAXA_APRENDIZAGEM * random.normalvariate(0, 1))

    return min(max(new_sigma, EPSILON_0), EPSILON_1)

In [875]:
def test_success(old_indiv_fitness, new_indiv_fitness):
    global CURR_SUCCESS

    if new_indiv_fitness < old_indiv_fitness:
        CURR_SUCCESS += 1

In [876]:
def get_success() -> float:
    global CURR_SUCCESS
    
    return CURR_SUCCESS / QTD_DE_FILHOS

In [877]:
def reset_success() -> None:
    global CURR_SUCCESS

    CURR_SUCCESS = 0

In [878]:
def mutate_and_calc_sucess(individuo: List[float], sigma: float):
    novo_sigma = calc_sigma(sigma)

    old_fitness = fitness(FITNESS_FUNC, individuo)
    
    for i in range(len(individuo)):
        individuo[i] = min(max(individuo[i] + novo_sigma * random.normalvariate(0, 1), -15), 15)

    new_fitness = fitness(FITNESS_FUNC, individuo)
    test_success(old_fitness, new_fitness)

    return individuo, novo_sigma

In [879]:
def mutate(filhos: List[Tuple[List[float], float]]):
    global QTD_DE_FILHOS

    filhos_mutados = []
    for i in range(QTD_DE_FILHOS):
        filhos_mutados.append(mutate_and_calc_sucess(filhos[i][0], filhos[i][1]))

    return filhos_mutados

### Seleção de sobreviventes

#### Geracional

In [880]:
def generational(filhos: List[Tuple[List[float], float]]):
    global POPULACAO, FITNESS_FUNC, TAM_DA_POPULACAO
    reset_population()
    
    filhos.sort(key=lambda filho: fitness(FITNESS_FUNC, filho[0]), reverse=False)
    POPULACAO = filhos[:TAM_DA_POPULACAO]

In [881]:
def elitista(filhos: List[Tuple[List[float], float]]):
    global POPULACAO, FITNESS_FUNC, TAM_DA_POPULACAO
    
    POPULACAO += filhos
    POPULACAO.sort(key=lambda filho: fitness(FITNESS_FUNC, filho[0]), reverse=False)
    POPULACAO = POPULACAO[:TAM_DA_POPULACAO]

### Loop

In [882]:
def is_close(a: float, b: float, diff: float = 0.00001):
    if a + diff > b and b + diff > a:
        return True
        
    return False

In [883]:
def should_stop(curr_best_fitness: float, curr_iter: int):
    if is_close(curr_best_fitness, 0):
        return True
    if curr_iter == MAX_ITERS:
        return True
    
    return False

In [885]:
reset_population()
generate_population()
iteration = 0

best_fitness = float("inf")

while (not should_stop(best_fitness, iteration)):
    reset_success()
    
    filhos = crossover()    
    filhos = mutate(filhos)
    
    success = get_success()

    for i in range(len(filhos)):
        filhos[i] = (filhos[i][0], regra_1_quinto(filhos[i][1], success))


    generational(filhos)

    curr_best_fitness = fitness(FITNESS_FUNC, POPULACAO[0][0])

    if curr_best_fitness <= best_fitness:
        best_fitness = curr_best_fitness
      
    if iteration % 100 == 0:
        print(best_fitness, POPULACAO[0][0], f'{POPULACAO[0][1] :.3f}')

    iteration += 1


print(iteration)


10451.757898046424 [2.770467975717234, 0.655786722316031, 11.087126441987731, -8.034719472485582, 5.713689795571851, 6.801195416660309, 1.085152114828619, -9.740664541747336, -9.3874340955719, 0.21340789754433737, 2.6076215299940793, -0.2558563293756068, 4.972164154652314, 4.5354638862588805, -5.221530964073491, 1.2655051675572335, 0.37260189333765886, 0.9997442698211838, 2.089818548723466, -0.6966808617457607, 1.5958662852282703, -2.707322810107274, -9.04249947084474, 10.601307624985926, -1.9715022341260948, 0.10134326379422194, 4.755641565139291, 4.599956079035522, 1.7015050993273915, -3.6734763794460776] 0.001
17.191521388787258 [-2.654286058377947, 1.5616493178483624, 0.42377343734658834, -0.48041664517799393, -0.15795759458565722, 0.4462531912745975, 0.21309310911418938, -0.2444454922587033, -0.058379250408827216, -0.24189300246831555, 0.030236713243779467, 0.05504434951922622, 0.010243525299809472, -0.007823749444079688, 0.006796168095756547, 0.06089617186518373, -0.0639082192869

# Estratégia Genética

## Versão 1